# 数据格式转换 and 预实验

本notebook目标是将txt格式的instruction数据转化成可以提供给模型训练的arrow格式，并用含100条数据的arrow格式训练模型，看看效果。

In [1]:
import re
import pandas as pd

# 4打开文件
with open('./generated_data_text.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

# 使用正则表达式提取 Instruction、Input 和 Output
pattern = r"Instruction:(.*?)\nInput:(.*?)\nOutput:(.*?)(?:\n\n|$)"
matches = re.findall(pattern, text, re.DOTALL)

# 将提取的数据转换为 DataFrame
df = pd.DataFrame(matches, columns=["Instruction", "Input", "Output"])


注意冒号是中文还是英文，这会影响正则表达式

In [2]:
len(df)

100

In [14]:
# 也可以直接用dataframe, 不需要转化为arrow格式
from datasets import Dataset
dataset = Dataset.from_pandas(df)

print(dataset)

Dataset({
    features: ['Instruction', 'Input', 'Output'],
    num_rows: 100
})


直接用Dataset.from_pandas()得到是Dataset对象，不是DatasetDict对象，因此不包含train, test等属性。   
   
接下来把Dataset对象转化为DatasetaDict对象，这样就可以跳过arrow文件直接得到合适的训练数据。

In [16]:
from datasets import DatasetDict

true_dataset = DatasetDict({
    'train': dataset
})

In [17]:
true_dataset

DatasetDict({
    train: Dataset({
        features: ['Instruction', 'Input', 'Output'],
        num_rows: 100
    })
})

数据的标准处理流程：
1. alpaca方法生成数据
2. 用正则表达式对数据进行切割，放到dataframe里面
3. 用Dataset.from_pandas()方法从dataframe得到Dataset对象
4. 用DatasetDict()将Dataset对象转换为DatasetDict对象，从而包含train, test等属性


1.22 今天能做到把txt文件转换